- Everything is structured output
- Key idea is to facilitate HCI by the Agent iterating on the target artifact as well as responding to the user
- User can always message using text or media
- Producing each part of the target artifact is a conversation focusing on different inputs

In [ ]:
class CourseMakerParameters(BaseModel):
    textbook: Media[ContentType.PDF]
    objective: Text

class CourseMakerTasks(StrEnum):
    LEARNING_OBJECTIVES = "learning_objectives"
    TITLE = "title"
    SYNOPSIS = "synopsis"


with Flow(name="course_maker", parameter_model=CourseMakerParameters, tasks=CourseMakerTasks) as flow:
    learning_objectives = GenerateLearningObjectives(name=CourseMakerTasks.LEARNING_OBJECTIVES, max_objectives=10)(
        source_material=flow.parameters.textbook, course_objective=flow.parameters.objective
    )
    title = GenerateTitle(name=CourseMakerTasks.TITLE, max_words=5)(learning_objectives=learning_objectives)
    synopsis = GenerateSynopsis(name=CourseMakerTasks.SYNOPSIS, max_words=50)(learning_objectives=learning_objectives, title=title)


with open("somatosensory.pdf", "rb") as f:
    pdf = f.read()

flow.compile(
    arguments=CourseMakerParameters(
        textbook=Media(pdf),
        objective=Text("Learn about the somatosensory system")
    ),
    unlocked=[CourseMakerTasks.TITLE, CourseMakerTasks.SYNOPSIS],
)
await flow.run()

In [ ]:
await flow.run_selected(tasks=[title, synopsis])

In [ ]:
class Flow[ParametersT: BaseModel, TasksT: ]